In [1]:
import pandas as pd
optimal_value = pd.read_csv('input/snw0501_sharedistance_530_90.csv') # 
optimal_value.head()

,i,j,i_detour,j_detour,case,i_O,i_D,j_O,j_D,i_t,j_t,savedistance,sharedistance
0,102,0,2614.690404,1786.734160,1,1547.0,1311.0,194.0,2400.0,2017-05-01 07:56:35,2017-05-01 07:57:41,-3094.079362,1307.345202
1,3,1426,1663.535721,0.000000,2,2245.0,194.0,1135.0,2665.0,2017-05-01 08:20:19,2017-05-01 08:21:05,1622.849047,3286.384768
2,950,8,1459.302527,833.022769,1,2178.0,3269.0,487.0,3694.0,2017-05-01 08:39:49,2017-05-01 08:40:00,-1146.162648,1146.162648
3,427,16,673.353874,2842.918560,1,1738.0,798.0,823.0,2610.0,2017-05-01 08:19:02,2017-05-01 08:20:21,-2043.371764,1472.900671
4,1724,19,948.587845,2705.898693,1,1738.0,487.0,1782.0,2665.0,2017-05-01 08:16:57,2017-05-01 08:17:13,-1499.901313,2154.585225


In [6]:
print(optimal_value.columns)
print(optimal_value['savedistance'].sum())
import numpy as np
print(optimal_value['i_detour'].mean())
print(optimal_value['j_detour'].mean())
print((optimal_value['i_detour'].mean() + optimal_value['j_detour'].mean()) / 2)
print(optimal_value['sharedistance'].mean())

Index(['i', 'j', 'i_detour', 'j_detour', 'case', 'i_O', 'i_D', 'j_O', 'j_D',
       'i_t', 'j_t', 'savedistance', 'sharedistance'],
      dtype='object')
51950.47064379618
1459.943473807913
1019.4208015387268
1239.6821376733199
2716.5810362772154


In [1]:
import pandas as pd
optimal_value = pd.read_csv('output/optimal_matching.csv')
optimal_value.head()

,Unnamed: 0,p0_id,p0_begin_time,p0_O_location,p0_D_location,p1_id,p1_begin_time,p1_O_location,p1_D_location
0,0,17592368144901,2017-05-01 08:47:28,1327.0,3404.0,17592368127779,2017-05-01 08:44:05,1355.0,3390.0
1,1,17592367729096,2017-05-01 08:14:43,3694.0,824.0,17592367709042,2017-05-01 08:09:43,3840.0,3303.0
2,2,17592367937835,2017-05-01 08:36:19,1309.0,1280.0,17592367973408,2017-05-01 08:31:42,185.0,776.0
3,3,17592367699325,2017-05-01 08:21:39,1351.0,1280.0,17592367947932,2017-05-01 08:25:58,539.0,291.0
4,4,17592367804398,2017-05-01 08:20:44,1350.0,3384.0,17592367674373,2017-05-01 08:19:40,1348.0,532.0


In [2]:
optimal_value[optimal_value['p0_id'] == optimal_value['p1_id']]

,Unnamed: 0,p0_id,p0_begin_time,p0_O_location,p0_D_location,p1_id,p1_begin_time,p1_O_location,p1_D_location


In [7]:
import Network as net


def is_fifo(p0, p1):
    fifo = [get_path(p1.O_location, p0.D_location),
                get_path(p0.D_location, p1.D_location)]
    lifo = [get_path(p1.O_location, p1.D_location),
                get_path(p1.D_location, p0.D_location)]
    if sum(fifo) < sum(lifo):
        return True, fifo
    else:
        return False, lifo


network = network = net.Network()
shortest_path = shortest_path = pd.read_csv('../input/shortest_path.csv')


def get_path(O, D):
    tmp = shortest_path[(shortest_path['O'] == O) & (
            shortest_path['D'] == D)]
    if tmp['distance'].unique():
        return tmp['distance'].unique()[0]
    else:
        return network.get_path(O, D)[0]



order_list = pd.read_csv('input/order.csv').sample(
            frac=0.3, random_state=1)

optimal_matching = pd.read_csv('output/optimal_matching.csv')
p0_id = optimal_matching['p0_id']
p1_id = optimal_matching['p1_id']
matched_results = {}
for i in range(len(p0_id)):
    matched_results[p0_id[i]] = p1_id[i]
    matched_results[p1_id[i]] = p0_id[i]

import Seeker
import numpy as np
Seekers = {}

for index, row in order_list.iterrows():
            seeker = Seeker.Seeker(row)
            seeker.set_shortest_path(get_path(
                seeker.O_location, seeker.D_location))
            # 生成一个正态分布随机数
            # 设置随机数生成的种子
            np.random.seed(index)
            seeker.waitingtime_threshold = max(
                30, np.random.normal(90, 10))
            Seekers[seeker.id] = seeker

saved_travel_distance = 0
detour_distance_lis = []
shared_distance_lis = []

for index, row in optimal_matching.iterrows():
    p0_id = row['p0_id']
    p1_id = row['p1_id']
    p0 = Seekers[p0_id]
    p1 = Seekers[p1_id]

    pickup_p1_distance = get_path(p0.O_location, p1.O_location)
    fifo, distance = is_fifo(p0, p1)
    if fifo:
            shared_distance = get_path(
                p1.O_location, p0.D_location)
            p0_invehicle = pickup_p1_distance + distance[0]
            p1_invehicle = sum(distance)
            p0_detour = p0_invehicle - \
                p0.shortest_distance
            p1_detour = p1_invehicle - \
                p1.shortest_distance

    else:
            shared_distance = p1.shortest_distance
            p0_invehicle = pickup_p1_distance + sum(distance)
            p1_invehicle = distance[0]
            p0_detour = p0_invehicle - \
                p0.shortest_distance
            p1_detour = p1_invehicle - \
                p1.shortest_distance

    distance_saving = p1.shortest_distance + p0.shortest_distance - (
            p0_invehicle + p1_invehicle - shared_distance)
    saved_travel_distance += distance_saving
    # 计算拼车距离
    shared_distance_lis.append(distance[0])
    # 计算绕行距离
    detour_distance_lis.append(p0_detour)
    detour_distance_lis.append(p1_detour)

In [8]:
print(saved_travel_distance)
print(np.mean(detour_distance_lis))
print(np.mean(shared_distance_lis))

1577626.9476525001
1387.1470057358465
6311.574163158915
